## Process for preparing the PlanetScope Image download.

Considerations:

 1. The image was ready downloaded from the PlanetScope repository and uploaded to Google Cloud Earth Engine Asset.
 2. This was done to avoid the high time in calculating the indexes it is more efficient to do this using geemap and/or python.

In [8]:
# Import the libraries
import geemap
import ee
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

In [5]:
ee.Authenticate() 


Successfully saved authorization token.


In [6]:
ee.Initialize()

In [2]:
#Nota: Acá debemos dibujar un área de interés. Sino en adelante nos fallará el código.

Map = geemap.Map(center = (11.2114, -15.1915), zoom = 12)
Map

Map(center=[11.2114, -15.1915], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

In [3]:
# Asegurarnos de la creación de una geometría / Por si no la crearon en el mapa anterior.
geometry = ee.Geometry.Polygon([[[-15.21471, 11.188674],
   [-15.21471, 11.222857],
   [-15.133365, 11.222857],
   [-15.133365, 11.188674],
   [-15.21471, 11.188674]]])

In [6]:
#@title GEE index's
collection = ee.ImageCollection("projects/planetscopejesus/assets/Datos_Gabo")

texture = ee.ImageCollection("projects/planetscopejesus/assets/Texture_cafine")
pca = ee.Image('projects/planetscopejesus/assets/pca_cafine').float()

collection = collection.filterDate('2022-05-01', '2022-05-30').filterBounds(geometry) 

# This function maps spectral indices for salinity Mapping using PlanetScope Imagery
def addIndices(img):
  # NDVI
  NDVI = img.normalizedDifference(['b8','b6']).rename('NDVI')
  SR = img.select('b8').divide(img.select('b6')).rename('SR')
  # GCVI
  GCVI = img.expression('(NIR/GREEN)-1',{'NIR':img.select('b8'),'GREEN':img.select('b3')}).rename('GCVI')
  # NDWI (GREEN-NIR)/(GREEN+NIR)
  NDWI = img.expression('(GREEN-NIR)/(GREEN+NIR)',{'NIR':img.select('b8'),'GREEN':img.select('b3')}).rename('NDWI')
  # VARI (Green−Red)/(Green+Reed−Blue)
  VARI = img.expression('(Green1-Red)/(Green1+Red-Blue)',{'Blue':img.select('b2'),'Red':img.select('b6'),'Green1':img.select('b3') }).rename('VARI')
  # GRVI (NIR/GREEN)
  GRVI = img.expression('(NIR/GREEN)',{'NIR':img.select('b8'),'GREEN':img.select('b3')}).rename('GRVI')
  # GNDVI (NIR-GREEN)/(NIR+GREEN)
  GNDVI = img.normalizedDifference(['b8','b3']).rename('GNDVI')
  # SAVI
  SAVI = img.expression('(NIR-RED)/((NIR+RED+0.5)*1.5)',{'NIR': img.select('b8'),'RED': img.select('b6')}).rename('SAVI')
  # VSSI = 2 ∗ 𝐵3 − 5(𝐵4 + 𝐵5) / Green(B3), Red(B4), NIR(B5)
  VSSI = img.expression('2*GREEN-5*(RED+NIR)',{'GREEN': img.select('b3'),'RED': img.select('b6'),'NIR': img.select('b8')}).rename('VSSI')
  # S1 (Blue/Red)
  S1 = img.select('b2').divide(img.select('b6')).rename('S1')
  # S1 (B − R)/(B + R)
  S2 = img.expression('(Blue-Red)/(Blue + Red)',{'Blue':img.select('b2'),'Red':img.select('b6') }).rename('S2')
  # S3_G1 (G1 × R)/B
  S3_G1 = img.expression('(Green1*Red)/(Blue)',{'Blue':img.select('b2'),'Red':img.select('b6'),'Green1':img.select('b3') }).rename('S3_G1')
  # S3_G2 (G1 × R)/B
  S3_G2 = img.expression('(Green2*Red)/(Blue)',{'Blue':img.select('b2'),'Red':img.select('b6'),'Green2':img.select('b4') }).rename('S3_G2')
  # S4 (B×R)**0.5
  S4 = img.expression('(Blue*Red)**0.5',{'Blue':img.select('b2'),'Red':img.select('b6')}).rename('S4')
  # S5_G1 ((B × R)/G1)
  S5_G1= img.expression('(Blue*Red)/Green1',{'Blue':img.select('b2'),'Green1':img.select('b3'),'Red':img.select('b6')}).rename('S5_G1')
  # S5_G2 ((B × R)/G1)
  S5_G2 = img.expression('(Blue*Red)/Green2',{'Blue':img.select('b2'),'Green2':img.select('b4'),'Red':img.select('b6')}).rename('S5_G2')
 # S6_G1 (R × NIR)/G1
  S6_G1 = img.expression('(Red*NIR)/Green1',{'NIR':img.select('b8'),'Green1':img.select('b3'),'Red':img.select('b6')}).rename('S6_G1')
  # S6_G2 (R × NIR)/G2
  S6_G2 = img.expression('(Red*NIR)/Green1',{'NIR':img.select('b8'),'Green1':img.select('b3'),'Red':img.select('b6')}).rename('S6_G2')
  # SI (B+R)**0.5
  SI = img.expression('(Blue+Red)**0.5',{'Blue':img.select('b2'),'Red':img.select('b6')}).rename('SI')
  # NDSI (R − NIR)/(R + NIR)
  NDSI = img.normalizedDifference(['b6','b8']).rename('NDSI')
  # SI1_G1 (G1×R)**0.5
  SI1_G1 = img.expression('(Green1*Red)**0.5',{'Green1':img.select('b3'),'Red':img.select('b6')}).rename('SI1_G1')
  # SI1_G2 (G1×R)**0.5
  SI1_G2 = img.expression('(Green2*Red)**0.5',{'Green2':img.select('b4'),'Red':img.select('b6')}).rename('SI1_G2')
  # SI2_G1 [(G1)**2 + (R)**2 + (NIR)**2]**0.5
  SI2_G1 = img.expression('((Green1)**2 + (Red)**2 + (NIR)**2)**0.5',{'NIR':img.select('b8'),'Green1':img.select('b3'),'Red':img.select('b6')}).rename('SI2_G1')
  # SI2_G2 [(G2)**2 + (R)**2 + (NIR)**2]**0.5
  SI2_G2 = img.expression('((Green2)**2 + (Red)**2 + (NIR)**2)**0.5',{'NIR':img.select('b8'),'Green2':img.select('b4'),'Red':img.select('b6')}).rename('SI2_G2')
  # SI3_G1 ((G1)**2 + (R)**2)**0.5
  SI3_G1 = img.expression('((Green1)**2 + (Red)**2)**0.5',{'Green1':img.select('b3'),'Red':img.select('b6')}).rename('SI3_G1')
  # SI3_G2 ((G1)**2 + (R)**2)**0.5
  SI3_G2 = img.expression('((Green2)**2 + (Red)**2)**0.5',{'Green2':img.select('b4'),'Red':img.select('b6')}).rename('SI3_G2')
  # Int1_G1 (G1 + R)/2
  Int1_G1 = img.expression('(Green1+Red)/2',{'Green1':img.select('b3'),'Red':img.select('b6')}).rename('Int1_G1')
  # Int1_G2 (G2 + R)/2
  Int1_G2 = img.expression('(Green2+Red)/2',{'Green2':img.select('b4'),'Red':img.select('b6')}).rename('Int1_G2')
  # Int2_G1 (G1 + R + NIR)/2
  Int2_G1 = img.expression('(Green1 + Red + NIR)/2',{'NIR':img.select('b8'),'Green1':img.select('b3'),'Red':img.select('b6')}).rename('Int2_G1')
  # Int2_G2 (G1 + R + NIR)/2
  Int2_G2 = img.expression('(Green2 + Red + NIR)/2',{'NIR':img.select('b8'),'Green2':img.select('b4'),'Red':img.select('b6')}).rename('Int2_G2')
  YRS6_G1 = img.expression('(Y*NIR)/G1',{'Y': img.select('b5'),'NIR': img.select('b8'),'G1': img.select('b3')}).rename('YRS6_G1')
  YRS6_G2 = img.expression('(Y*NIR)/G2',{'Y': img.select('b5'),'NIR': img.select('b8'), 'G2': img.select('b4')}).rename('YRS6_G2')
  RS6_G1 = img.expression('(Red_edge*NIR)/G1',{'Red_edge': img.select('b7'),'NIR': img.select('b8'),'G1': img.select('b3')}).rename('RS6_G1')
  RS6_G2 = img.expression('(Red_edge*NIR)/G2',{'Red_edge': img.select('b7'),'NIR': img.select('b8'),'G2': img.select('b4')}).rename('RS6_G2')
  RS1 = img.expression('B/Red_edge', {'B': img.select('b2'), 'Red_edge': img.select('b7')}).rename('RS1')
  RS2 = img.expression('(B - Red_edge) / (B + Red_edge)', {'B': img.select('b2'), 'Red_edge': img.select('b7')}).rename('RS2')
  RS3_G1 = img.expression('(G1 * Red_edge) / B', {'G1': img.select('b3'), 'Red_edge': img.select('b7'), 'B': img.select('b2')}).rename('RS3_G1')
  RS3_G2 = img.expression('(G2 * Red_edge) / B', {'G2': img.select('b4'), 'Red_edge': img.select('b7'), 'B': img.select('b2')}).rename('RS3_G2')
  RS4 = img.expression('(B * Red_edge) ** 0.5', {'B': img.select('b2'), 'Red_edge': img.select('b7')}).rename('RS4')
  RS5_G1 = img.expression('(B * Red_edge) / G1', {'B': img.select('b2'), 'Red_edge': img.select('b7'), 'G1': img.select('b3')}).rename('RS5_G1')
  RS5_G2 = img.expression('(B * Red_edge) / G2', {'B': img.select('b2'), 'Red_edge': img.select('b7'), 'G2': img.select('b4')}).rename('RS5_G2')
  RS6_G1 = img.expression('(Red_edge * NIR) / G1', {'Red_edge': img.select('b7'), 'NIR': img.select('b8'), 'G1': img.select('b3')}).rename('RS6_G1')
  RS6_G2 = img.expression('(Red_edge * NIR) / G2', {'Red_edge': img.select('b7'), 'NIR': img.select('b8'), 'G2': img.select('b4')}).rename('RS6_G2')
  RNDSI = img.expression('(Red_edge - NIR) / (Red_edge + NIR)', {'Red_edge': img.select('b7'), 'NIR': img.select('b8')}).rename('RNDSI')
  RNDVI = img.expression('(NIR - Red_edge) / (NIR + Red_edge)', {'Red_edge': img.select('b7'), 'NIR': img.select('b8')}).rename('RNDVI')
  RSI = img.expression('(B + Red_edge) ** 0.5', {'B': img.select('b2'), 'Red_edge': img.select('b7')}).rename('RSI')
  RSI1_G1 = img.expression('(G1 * Red_edge) ** 0.5', {'G1': img.select('b3'), 'Red_edge': img.select('b7')}).rename('RSI1_G1')
  RSI1_G2 = img.expression('(G2 * Red_edge) ** 0.5', {'G2': img.select('b4'), 'Red_edge': img.select('b7')}).rename('RSI1_G2')
  RSI2_G1 = img.expression('((G1) ** 2 + (Red_edge) ** 2 + (NIR) ** 2) ** 0.5', {'G1': img.select('b3'), 'Red_edge': img.select('b7'), 'NIR': img.select('b8')}).rename('RSI2_G1')
  RSI2_G2 = img.expression('((G2) ** 2 + (Red_edge) ** 2 + (NIR) ** 2) ** 0.5', {'G2': img.select('b4'), 'Red_edge': img.select('b7'), 'NIR': img.select('b8')}).rename('RSI2_G2')
  RSI3_G1 = img.expression('((G1) ** 2 + (Red_edge) ** 2) ** 0.5', {'G1': img.select('b3'), 'Red_edge': img.select('b7')}).rename('RSI3_G1')
  RSI3_G2 = img.expression('((G2) ** 2 + (Red_edge) ** 2) ** 0.5', {'G2': img.select('b4'), 'Red_edge': img.select('b7')}).rename('RSI3_G2')
  RInt1_G1 = img.expression('(G1 + Red_edge) / 2', {'G1': img.select('b3'), 'Red_edge': img.select('b7')}).rename('RInt1_G1')
  RInt1_G2 = img.expression('(G2 + Red_edge) / 2', {'G2': img.select('b4'), 'Red_edge': img.select('b7')}).rename('RInt1_G2')
  RInt2_G1 = img.expression('(G1 + Red_edge + NIR) / 2', {'G1': img.select('b3'), 'Red_edge': img.select('b7'), 'NIR': img.select('b8')}).rename('RInt2_G1')
  RInt2_G2 = img.expression('(G2 + Red_edge + NIR) / 2', {'G2': img.select('b4'), 'Red_edge': img.select('b7'), 'NIR': img.select('b8')}).rename('RInt2_G2')
  ####################################################################################### Table 4 ######################################################################################
  YRS1 = img.expression('B/Y', {'B': img.select('b2'), 'Y': img.select('b5')}).rename('YRS1')
  YRS2 = img.expression('(B - Y) / (B + Y)', {'B': img.select('b2'), 'Y': img.select('b5')}).rename('YRS2')
  YRS3_G1 = img.expression('(G1 * Y) / B', {'G1': img.select('b3'), 'Y': img.select('b5'), 'B': img.select('b2')}).rename('YRS3_G1')
  YRS3_G2 = img.expression('(G2 * Y) / B', {'G2': img.select('b4'), 'Y': img.select('b5'), 'B': img.select('b2')}).rename('YRS3_G2')
  YRS4 = img.expression('(B * Y) ** 0.5', {'B': img.select('b2'), 'Y': img.select('b5')}).rename('YRS4')
  YRS5_G1 = img.expression('(B * Y) / G1', {'B': img.select('b2'), 'Y': img.select('b5'), 'G1': img.select('b3')}).rename('YRS5_G1')
  YRS5_G2 = img.expression('(B * Y) / G2', {'B': img.select('b2'), 'Y': img.select('b5'), 'G2': img.select('b4')}).rename('YRS5_G2')
  YRS6_G1 = img.expression('(Y * NIR) / G1', {'Y': img.select('b5'), 'NIR': img.select('b8'), 'G1': img.select('b3')}).rename('YRS6_G1')
  YRS6_G2 = img.expression('(Y * NIR) / G2', {'Y': img.select('b5'), 'NIR': img.select('b8'), 'G2': img.select('b4')}).rename('YRS6_G2')
  YRSI = img.expression('(B + Y) ** 0.5', {'B': img.select('b2'), 'Y': img.select('b5')}).rename('YRSI')
  YRSI1_G1 = img.expression('(G1 * Y) ** 0.5', {'G1': img.select('b3'), 'Y': img.select('b5')}).rename('YRSI1_G1')
  YRSI1_G2 = img.expression('(G2 * Y) ** 0.5', {'G2': img.select('b4'), 'Y': img.select('b5')}).rename('YRSI1_G2')
  YRSI2_G1 = img.expression('(((G1) ** 2) + ((Y) ** 2) + ((NIR) ** 2)) ** 0.5', {'G1': img.select('b3'), 'Y': img.select('b5'), 'NIR': img.select('b8')}).rename('YRSI2_G1')
  YRSI2_G2 = img.expression('(((G2) ** 2) + ((Y) ** 2) + ((NIR) ** 2)) ** 0.5', {'G2': img.select('b4'), 'Y': img.select('b5'), 'NIR': img.select('b8')}).rename('YRSI2_G2')
  YRSI3_G1 = img.expression('(((G1) ** 2) + ((Y) ** 2)) ** 0.5', {'G1': img.select('b3'), 'Y': img.select('b5')}).rename('YRSI3_G1')
  YRSI3_G2 = img.expression('(((G2) ** 2) + ((Y) ** 2)) ** 0.5', {'G2': img.select('b4'), 'Y': img.select('b5')}).rename('YRSI3_G2')
  YRInt1_G1 = img.expression('(G1 + Y) / 2', {'G1': img.select('b3'), 'Y': img.select('b5')}).rename('YRInt1_G1')
  YRInt1_G2 = img.expression('(G2 + Y) / 2', {'G2': img.select('b4'), 'Y': img.select('b5')}).rename('YRInt1_G2')
  YRInt2_G1 = img.expression('(G1 + Y + NIR) / 2', {'G1': img.select('b3'), 'Y': img.select('b5'), 'NIR': img.select('b8')}).rename('YRInt2_G1')
  YRInt2_G2 = img.expression('(G2 + Y + NIR) / 2', {'G2': img.select('b4'), 'Y': img.select('b5'), 'NIR': img.select('b8')}).rename('YRInt2_G2')
  YRNDSI = img.expression('(Y - NIR) / (Y + NIR)', {'Y': img.select('b5'), 'NIR': img.select('b8')}).rename('YRNDSI')
  ##########################################################
  YRNDVI = img.expression('(NIR - Y) / (NIR + Y)', {'NIR': img.select('b8'), 'Y': img.select('b5')}).rename('YRNDVI')
  YBS1 = img.expression('Y / R', {'Y': img.select('b5'), 'R': img.select('b6')}).rename('YBS1')
  YBS2 = img.expression('(Y - R) / (Y + R)', {'Y': img.select('b5'), 'R': img.select('b6')}).rename('YBS2')
  YBS4 = img.expression('(Y * R) ** 0.5', {'Y': img.select('b5'), 'R': img.select('b6')}).rename('YBS4')
  YBS5_G1 = img.expression('(Y * R) / G1', {'Y': img.select('b5'), 'R': img.select('b6'), 'G1': img.select('b3')}).rename('YBS5_G1')
  YBS5_G2 = img.expression('(Y * R) / G2', {'Y': img.select('b5'), 'R': img.select('b6'), 'G2': img.select('b4')}).rename('YBS5_G2')
  YBSI = img.expression('(Y + R) ** 0.5', {'Y': img.select('b5'), 'R': img.select('b6')}).rename('YBSI')
  YGS3 = img.expression('(Y * R) / B', {'B': img.select('b2'), 'Y': img.select('b5'), 'R': img.select('b6')}).rename('YGS3')
  YGSI1 = img.expression('(Y * R) ** 0.5', {'R': img.select('b6'), 'Y': img.select('b5')}).rename('YGSI1')
  YGSI2 = img.expression('((Y ** 2) + (R ** 2) + (NIR ** 2)) ** 0.5', {'R': img.select('b6'), 'Y': img.select('b5'), 'NIR': img.select('b8')}).rename('YGSI2')
  YGSI3 = img.expression('((Y ** 2) + (R ** 2)) ** 0.5', {'Y': img.select('b5'), 'R': img.select('b6')}).rename('YGSI3')
  YGInt1 = img.expression('(Y + R) / 2', {'R': img.select('b6'), 'Y': img.select('b5')}).rename('YGInt1')
  YGInt2 = img.expression('(Y + R + NIR) / 2', {'R': img.select('b6'), 'Y': img.select('b5'), 'NIR': img.select('b8')}).rename('YGInt2')
  YNS6_G1 = img.expression('(R * Y) / G1', {'R': img.select('b6'), 'Y': img.select('b5'), 'G1': img.select('b3')}).rename('YNS6_G1')
  YNS6_G2 = img.expression('(R * Y) / G2', {'R': img.select('b6'), 'Y': img.select('b5'), 'G2': img.select('b4')}).rename('YNS6_G2')
  YNSI2_G1 = img.expression('((G1 ** 2) + (R ** 2) + (Y ** 2)) ** 0.5', {'G1': img.select('b3'), 'Y': img.select('b5'), 'R': img.select('b6')}).rename('YNSI2_G1')
  YNSI2_G2 = img.expression('((G2 ** 2) + (R ** 2) + (Y ** 2)) ** 0.5', {'G2': img.select('b4'), 'Y': img.select('b5'), 'R': img.select('b6')}).rename('YNSI2_G2')
  YNInt2_G1 = img.expression('(G1 + R + Y) / 2', {'R': img.select('b6'), 'Y': img.select('b5'), 'G1': img.select('b3')}).rename('YNInt2_G1')
  YNInt2_G2 = img.expression('(G2 + R + Y) / 2', {'R': img.select('b6'), 'Y': img.select('b5'), 'G2': img.select('b4')}).rename('YNInt2_G2')
  YNNDSI = img.expression('(R - Y) / (R + Y)', {'Y': img.select('b5'), 'R': img.select('b6')}).rename('YNNDSI')
  YNNDVI = img.expression('(Y - R) / (Y + R)', {'R': img.select('b6'), 'Y': img.select('b5')}).rename('YNNDVI')
  return img \
    .addBands(
[
    NDVI, SR, GCVI, NDWI, VARI, GNDVI, GRVI, SAVI, VSSI, S1, S2, S3_G1, S3_G2, S4, S5_G1, S5_G2,
    S6_G1, S6_G2, SI, NDSI, SI1_G1, SI1_G2, SI2_G1, SI2_G2, SI3_G1, SI3_G2, Int1_G1, Int1_G2,
    Int2_G1, Int2_G2, YRS6_G1, YRS6_G2, RS6_G1, RS6_G2, RS1, RS2, RS3_G1, RS3_G2, RS4, RS5_G1,
    RS5_G2, RNDSI, RNDVI, RSI, RSI1_G1, RSI1_G2, RSI2_G1, RSI2_G2, RSI3_G1, RSI3_G2, RInt1_G1,
    RInt1_G2, RInt2_G1, RInt2_G2,YRS1, YRS2, YRS3_G1, YRS3_G2, YRS4, YRS5_G1, YRS5_G2, YRSI, YRSI1_G1, 
    YRSI1_G2, YRSI2_G1, YRSI2_G2, YRSI3_G1, YRSI3_G2, YRInt1_G1, YRInt1_G2, YRInt2_G1, YRInt2_G2, YRNDSI,
    YRNDVI, YBS1, YBS2, YBS4, YBS5_G1, YBS5_G2, YBSI, YGS3, YGSI1, YGSI2, YGSI3, YGInt1,
     YGInt2, YNS6_G1, YNS6_G2, YNSI2_G1, YNSI2_G2, YNInt2_G1, YNInt2_G2, YNNDSI, YNNDVI

]

    )



ps = collection

#Add the indices
ps_1 = ps.map(addIndices)
composite = ps_1 \
              .mean()\
              .float()

texture1 = texture.mean()


# Seleccionar la banda NIR, for the type of calculation shoud be in Int32, because glcmTexture only admit these type of data
nir = composite.select('b8').toInt32()

# Calcular la textura utilizando el índice de contraste (Contrast Index)
texture = nir.glcmTexture()
texture_b8 = texture.select('b8_asm').float()


composite = composite.addBands(texture1).addBands(pca).addBands(texture_b8)

# Agregar la capa de textura al mapa
Map.addLayer(texture, {}, 'Textura NIR')
Map.addLayer(composite, {'bands': ['b6',  'b4',  'b2'], 'min': 201, 'max': 2464}, 'RGB')
Map.addLayer(composite, {'bands': ['S1',  'S2',  'S3_G1'], 'min': 0, 'max': 1}, 'Index')

# NOTA RELEVANTE sobre orden raster de textura de suelo caso Cafine:
# B1_1 = Clay + Silt
# B2_1 = Sand
# B3_1 = Silt
# B4_1 = Clay

# 'RNDSI', 'NDSI', 'NDWI','b3_1 (Silt Raster)','b4_1 (Clay Raster'
#Exportar las bandas para modelar Cafine.
export = composite.select(['RNDSI', 'NDWI','NDSI','b3_1'])

Run the next line. You can see the process of export in: https://code.earthengine.google.com/ or in https://code.earthengine.google.com/tasks

In [7]:
geemap.ee_export_image_to_drive(
    export, description='PS_Index_filtered', folder='ML_Final_Work', region=geometry, scale= 3, crs = 'EPSG:2095'
)

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

credentials_path = 'C:/Users/cespe/client_secret_775448020170-glgpi5v37ur14of7spbms2gi67g7silt.apps.googleusercontent.com.json'

# Autenticar y crear la instancia de PyDrive
gauth = GoogleAuth()
gauth.LoadClientConfigFile(credentials_path)
gauth.LocalWebserverAuth()  # Abre una ventana del navegador para autenticarse

drive = GoogleDrive(gauth)



In [41]:
# ID de la carpeta en Google Drive
folder_id = '1gFgdT90qRKu4pFW-fVbKagEzPrkKTSDb'  # Reemplaza con el ID real de tu carpeta
file_title = 'PS_Index_filtered.tif'  # Nombre del archivo que buscas

try:
    file_list = drive.ListFile({'q': f"'{folder_id}' in parents and trashed=false"}).GetList()
    for file in file_list:
        if file['title'] == file_title:
            file.GetContentFile(file_title)  # Descargar el archivo
            print(f"Archivo {file_title} descargado exitosamente.")
            break
    else:
        print(f"Archivo {file_title} no encontrado en la carpeta {folder_id}.")
except Exception as e:
    print(f"Error al buscar el archivo en Google Drive: {e}")


Archivo PS_Index_filtered.tif descargado exitosamente.


## Buffer for image statistics extraction and correlation with sodium absorption rate (SAR)